#Assignment 3: Brain Tumour Detection
## Team: Strykers
*   Sotero Jr Saberon
*   Leon Ambrose Lusbo
*   Norah Albaqami






The following methods are used:

* Used a pretrained <b>Mask R-CNN with ResNeXt-101-32x8d for Feature Pyramid Network</b> from detectron2 [1,2]
* Modified the official notebook [3]
* Used validation and test sets for testing,

## Dataset information
TumorSegmentation
===================================================================

This dataset was exported via roboflow.com on August 19, 2023

The dataset includes 2146 images.
Tumors are annotated in COCO Segmentation format.
====================================================================

The following pre-processing was applied to each image:
* Auto-orientation of pixel data (with EXIF-orientation stripping)
* Resize to 640x640 (Stretch)
* No image augmentation techniques were applied.
====================================================================

Provided by Roboflow
License: CC BY 4.0


## References
1. Ren, S., He, K., Girshick, R., & Sun, J. (2015). Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks (Version 3). arXiv. https://doi.org/10.48550/ARXIV.1506.01497
2. https://detectron2.readthedocs.io/en/latest/
3. https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5

#Step 1: Mount the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Step 2: Import Libraries

In [ ]:
import sys, os, distutils.core
from IPython.display import clear_output

!python -m pip install pyyaml==5.1
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))
clear_output()

In [ ]:
# Importing dependencies

import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
print("detectron2 version:", detectron2.__version__)

import numpy as np
import os, json, cv2, random
from IPython import display
import PIL
import matplotlib.pyplot as plt

# importing detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


SEED = 99
THRESHOLD = 0.6

detectron2 version: 0.6


# Step 3: Data Collection/Ingestion

# Step 4: Model Training

## Load Config

## Save the Config

## Save the Model

# Step 7: Display Training Results

# Step 8: Model Evaluation

# Step 9: Model Testing

#Step 10. Visualize the Model Architecture